In [62]:
import pandas as pd
import numpy as np

In [64]:
df0 = pd.read_csv("Noah_5Years_000.csv", skiprows=4)
df1 = pd.read_csv("Noah_5Years_002.csv", skiprows=4)
df2 = pd.read_csv("Noah_5Years_003.csv", skiprows=4)

df = pd.concat([df0, df1, df2])
df = df[['Timestamp', 'Noah - Water Use (gal)']]
df[['Date', 'Time']] = df['Timestamp'].str.split(' ', expand = True)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day_name()
df = df[['Year', 'Month', 'Day', 'Time', 'Noah - Water Use (gal)']]

year_bool_array = (df['Year'] >= 2021) | (df['Year'] <= 2019)
month_bool_array = ((df['Month'] >= 9) & (df['Month'] <= 12)) | ((df['Month'] >= 2) & (df['Month'] <= 5))

df = df[year_bool_array & month_bool_array]
df = df[['Day', 'Time', 'Noah - Water Use (gal)']]
df.columns = ['Day', 'Time', 'Water']
df.dropna(inplace=True)
df


,Day,Time,Water
2974,Thursday,00:00,2.90586
2975,Thursday,00:15,1.52196
2976,Thursday,00:30,5.70775
2977,Thursday,00:45,15.56600
2978,Thursday,01:00,5.39654
...,...,...,...
25069,Saturday,22:45,0.86475
25070,Saturday,23:00,3.35533
25071,Saturday,23:15,2.83638
25072,Saturday,23:30,0.89934


In [65]:
df.isnull().any()

Day      False
Time     False
Water    False
dtype: bool

In [66]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

X = df.drop('Water', axis=1)
y = df['Water']

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train_transformed = OneHotEncoder().fit_transform(X_train)
X_test_transformed = OneHotEncoder().fit_transform(X_test)


In [67]:
rgs = MLPRegressor(hidden_layer_sizes=(5,))
rgs.fit(X_train_transformed, y_train)
y_pred = rgs.predict(X_test_transformed)

c:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [68]:
pd.Series(y_pred)

0        20.880911
1        17.714989
2        22.866929
3        24.811276
4         0.876933
           ...    
19501     7.877748
19502    17.877605
19503    17.663059
19504    19.547326
19505    24.932014
Length: 19506, dtype: float64

In [69]:
y_test

36137    33.96848
2608     27.43037
45687     8.99380
34920    27.39698
20581     4.15110
           ...   
44104     3.45910
25841    23.41813
49921    15.32427
4662     37.32411
45233    27.84595
Name: Water, Length: 19506, dtype: float64

In [70]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.2661998795261592